In [122]:
import jax
import jax.numpy as jnp
from jax import random, grad, jit, jacfwd, jacrev
from jax.scipy.linalg import inv, svd, eigh, det
from jax.lax import scan
from scipy.linalg import solve_discrete_are, norm
from tqdm.auto import tqdm
import matplotlib.pyplot as plt
from jax.tree_util import Partial
from functools import partial
from jax_vi import KL_gaussian, log_likelihood, KL_sum, plot_optimization_results, plot_k_matrices

from jax_filters import apply_filtering_fixed_nonlinear, kalman_filter_process, filter_step_nonlinear
from jax_models import visualize_observations, Lorenz63, generate_true_states, generate_localization_matrix

N = 10 # number of Monte Carlo samples
num_steps = 250  # Number of time steps
J0 = 0
n = 3   # Number of state variables
key = random.PRNGKey(0)  # Random key for reproducibility
Q = 0.3 * jnp.eye(n)  # Process noise covariance
R = 0.01 * jnp.eye(n)  # Observation noise covariance
H = jnp.eye(n) # Observation matrix (identity matrix for direct observation of all state variables)

n_ensemble = 20
observation_interval = 1
initial_state = random.normal(random.PRNGKey(0), (n,)) 
m0 = initial_state
C0 = Q

l63_model = Lorenz63()
l63_step = Partial(l63_model.step)

jacobian_function = jacrev(l63_step, argnums=0)
jac_func = Partial(jacobian_function)
state_transition_function = l63_step

observations, true_states = generate_true_states(key, num_steps, n, initial_state, H, Q, R, l63_step, observation_interval)



In [144]:
def resample_particles(key, particles, weights):
    # Multinomial Resampling
    num_particles = particles.shape[0]
    cumulative_sum = jnp.cumsum(weights)
    indices = jnp.searchsorted(cumulative_sum, random.uniform(key, (num_particles,)))
    return particles[indices]

In [145]:
def systematic_resample(key, particles, weights):
    N = len(weights)
    offset = random.uniform(key, shape=(), minval=0, maxval=1)
    positions = (jnp.arange(N) + offset) / N
    
    indexes = jnp.zeros(N, dtype='int32')
    cumulative_sum = jnp.cumsum(weights)
    i, j = 0, 0
    while i < N:
        if positions[i] < cumulative_sum[j]:
            indexes = indexes.at[i].set(j)
            i += 1
        else:
            j += 1

    return particles[indexes]

In [ ]:
def update_weights(particles, observation, H, R):
    # Calculate the likelihood of each particle given the observation
    predicted_observations = vmap(lambda x: jnp.dot(H, x), in_axes=0, out_axes=0)(particles)
    obs_dim = observation.shape[0]
    inv_R = jnp.linalg.inv(R)
    diff = observation - predicted_observations
    likelihood = jnp.exp(-0.5 * vmap(lambda d: jnp.dot(d, jnp.dot(inv_R, d.T)), in_axes=0, out_axes=0)(diff))
    likelihood = likelihood / likelihood.sum()  # Normalize the weights
    return likelihood
    
def particle_filter(key, num_particles, num_steps, initial_state, observations, state_transition_function, H, Q, R, system = False):
    mean = jnp.tile(initial_state, (num_particles, 1))
    particles = random.multivariate_normal(key, mean, Q, shape=(num_particles,))
    step = vmap(state_transition_function, in_axes=0, out_axes=0)
    ensemble = []
    for t in range(num_steps):
        key, subkey = random.split(key)
        particles = step(particles) + random.multivariate_normal(key, jnp.zeros(particles.shape[1]), Q, shape=particles.shape[:1])
        if t % observation_interval == 0:
            observation = observations[t // observation_interval]
            weights = update_weights(particles, observation, H, R)
            if system:
                particles = systematic_resample(subkey, particles, weights)
            else:
                particles = resample_particles(subkey, particles, weights)
        ensemble.append(particles)
        

    # Transpose to (timestep, state_dim, num_particles)
    return jnp.transpose(jnp.array(ensemble), (0, 2, 1))

key = random.PRNGKey(0)  # Initialize PRNG key
num_particles = 20
initial_state = random.normal(random.PRNGKey(0), (n,)) 

estimated_states = particle_filter(key, num_particles, num_steps, initial_state, observations, state_transition_function, H, Q, R, True)


In [ ]:
from jax_filters import ensrf_steps, kalman_filter_process, ensrf_step
inflation = 1.5  # Fixed starting value for inflation
radius_opt = 1.2
N = 10  # Number of MC samples
m0 = initial_state
C0 = Q  # Initial covariance, assuming Q is your process noise covariance
ensemble_init = random.multivariate_normal(key, initial_state, Q, (n_ensemble,)).T
localization_matrix = generate_localization_matrix(n, radius_opt)

base_m, base_C, base_K = kalman_filter_process(l63_step, jac_func, m0, C0, observations, H, Q, R)

pred_states, pred_covariances, analysis_states, analysis_covariances = ensrf_steps(l63_step, n_ensemble, ensemble_init, num_steps, observations, observation_interval, H, Q, R, localization_matrix, inflation, key)

def calculate_covariance_matrix(states):
    mean = jnp.mean(states, axis=2)
    deviations = states - mean[:, :, jnp.newaxis]
    covariance_matrix = jnp.einsum('ijk,ilk->ijl', deviations, deviations) / (states.shape[2] - 1)
    return covariance_matrix

particle_cov = calculate_covariance_matrix(estimated_states)
frobenius_norms = [jnp.linalg.norm(base_C[i] - particle_cov[i], 'fro') for i in range(250)]




In [ ]:
analysis_covariances.shape

In [ ]:
pf_mean_estimates = jnp.mean(estimated_states, axis=2)
ensrf_mean_estimates = jnp.mean(analysis_states, axis=2)

# Compute errors as the absolute difference from true states
pf_errors = jnp.abs(pf_mean_estimates - true_states)**2
kf_errors = jnp.abs(base_m - true_states)**2
ensrf_errors = jnp.abs(ensrf_mean_estimates - true_states)**2

pf_mean_error = jnp.mean(pf_errors, axis=1)
kf_mean_error = jnp.mean(kf_errors, axis=1)
ensrf_mean_error = jnp.mean(ensrf_errors, axis=1)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(pf_mean_error, label='Particle Filter Mean Error', marker='o')
plt.plot(kf_mean_error, label='Kalman Filter Mean Error', marker='x')
plt.plot(ensrf_mean_error, label='EnSRF Mean Error', marker='s')
plt.title('Comparison of Filter Accuracy Over Time')
plt.xlabel('Time Step')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.grid(True)
plt.show()


In [118]:
base_m[:5]

Array([[ 1.9215502 , -0.59329814,  0.2814261 ],
       [ 1.772846  ,  0.1740785 , -0.01892382],
       [ 1.9457296 , -0.2977463 ,  1.0511189 ],
       [ 1.218002  ,  0.20786238,  0.5761704 ],
       [ 1.1740887 ,  0.42936862,  0.60618496]], dtype=float32)

In [121]:
analysis_states.mean(axis=2)[:5]

Array([[ 1.9251378 , -0.5991712 ,  0.28072774],
       [ 1.7729867 ,  0.17414775, -0.01907295],
       [ 1.9457743 , -0.2969916 ,  1.0502003 ],
       [ 1.2184255 ,  0.20807543,  0.57656044],
       [ 1.1740917 ,  0.42947647,  0.6061563 ]], dtype=float32)